# Library

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# [로드] 등록&제원 정보 2022.12.28 기준 

In [2]:
ods_fold = 'D:/data/big2/ODS'

In [3]:
# about 8m 42s
file_name = '[ODS]등록정보&제원정보(2022.12.29).csv'
df = pd.read_csv(os.path.join(ods_fold, file_name), encoding='cp949', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29534047 entries, 0 to 29534046
Data columns (total 49 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   현재회원번호        object 
 7   차대번호          object 
 8   최초등록일자        int64  
 9   차량연식          int64  
 10  차량제작일자        float64
 11  배출가스검사유효일자    float64
 12  제원관리번호        object 
 13  배출가스인증번호      object 
 14  배출가스등급        object 
 15  저공해조치YN(미사용)  object 
 16  저공해조치종류       float64
 17  저공해조치구조변경YN   object 
 18  차량등록번호숫자정보    int64  
 19  차량말소YN        object 
 20  최초회원번호        object 
 21  취득일자          object 
 22  취득금액          float64
 23  생성일시_등록정보     object 
 24  수정일시_등록정보     object 
 25  차명            object 
 26  차종_제원정보       object 
 27  차종분류          object 
 28  차종유형(소분류)     object 
 29  자동차형식         object 
 30  제작사명          ob

In [4]:
df['본거지법정동코드'].head()

0    1165010700
1    2711010500
2    2820010300
3    4213011200
4    4812110500
Name: 본거지법정동코드, dtype: int64

# [로드] 본거지법정동코드

In [5]:
fold = 'D:/data/big2'

In [6]:
file_name = '법정동코드_2022.10.14.xlsx'
code = pd.read_excel(os.path.join(fold, file_name))
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46340 entries, 0 to 46339
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     46340 non-null  int64  
 1   시도        46340 non-null  object 
 2   시구군       46317 non-null  object 
 3   읍면동       45847 non-null  object 
 4   리         33682 non-null  object 
 5   지역3       0 non-null      float64
 6   폐지여부      46340 non-null  object 
 7   대기관리권역    46340 non-null  object 
 8   대기관리권역YN  46334 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 3.2+ MB


In [7]:
code['법정동코드'].head()

0    1100000000
1    1111000000
2    1111010100
3    1111010200
4    1111010300
Name: 법정동코드, dtype: int64

# [merge] 법정동코드 기준

In [8]:
# '본거지법정도코드'열 데이터 확인
df['본거지법정동코드'].isnull().sum()

0

In [9]:
df.shape

(29534047, 49)

In [10]:
# about 39s
tot = df.merge(code, left_on='본거지법정동코드', right_on='법정동코드', how='left')
tot.shape

(29534047, 58)

In [11]:
len(tot['차대번호'].unique())

29521505

In [12]:
tot['차대번호'].isnull().sum()

0

# 전처리

## 말소제거
- 차량말소YN

In [13]:
# about 
tot1 = tot[tot['차량말소YN'] == 'N'].reset_index(drop=True)
tot1.shape

(25693532, 58)

In [14]:
tot1.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN'],
      dtype='object')

In [60]:
tot1_1 = tot1.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates(['차대번호']).reset_index(drop=True)
tot1_1.shape

(25692512, 58)

In [ ]:
tot1_1['배출가스등급'].value_counts(dropna=False)

## 폐지여부 확인

In [15]:
tot1['폐지여부'].value_counts(dropna=False)

존재     25611213
폐지        82317
NaN           2
Name: 폐지여부, dtype: int64

In [16]:
tot2 = tot1[tot1['폐지여부'] == '존재'].reset_index(drop=True)
tot2.shape

(25611213, 58)

## 법정동코드 확인

In [17]:
tot2['법정동코드'].isnull().sum()

0

## 연료 확인

In [18]:
tot2['연료'].isnull().sum()

146468

In [19]:
tot2['연료'].value_counts(dropna=False)

휘발유            12138821
경유              9855813
LPG(액화석유가스)     1899359
휘발유 하이브리드       1092740
전기               368150
NaN              146468
경유 하이브리드          35727
CNG(압축천연가스)       30372
수소                28199
LPG 하이브리드         14301
기타연료                926
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: 연료, dtype: int64

In [20]:
tot2.loc[tot2['연료'] == '기타연료', '엔진형식'].unique()

array([nan, '3304124', '08784250915', '09801220246', '10784203005',
       '10784203004', 'LNS', '0', '01821300581', 'G4AJ', 'G15CF', 'B5',
       'H20MR', 'G16SF', 'MSV3', 'ZB', '-', '1NZ', '2AZ', 'RW-300',
       'M1082', '.'], dtype=object)

In [33]:
tot2.loc[tot2['연료'].isnull() == True, '엔진형식'].unique()

array([nan, 'DA640', '10-78-11-64609', '077801946', 'D0846HM',
       '06744200580', '10772217991', 'DA120', '10811142167',
       '04784300942', '02791162726', '01744500006', '01-77-13-00969',
       'KG-69-C-D-201452', 'F-6', 'GJS(LJS)', '10781153612',
       '03781260209', '3.8', '0', 'G4MBG', 'G4AJF045570', 'LNS',
       'CY250909', '834C.146', 'G4CP', '31L', '20EV', '41.5', 'B', 'mx',
       '102 962', 'VQ30', 'K04D3', 'V6', 'G15MF', 'F8A', 'CR8.13.1',
       '40S1', 'F8C', 'B-123', '5.0HD', 'M28/16', 'M30-69', 'CRJ', 'EFA',
       '104 941', '3S-GTE', '111 970', 'M64/02', '117.986', '2S25',
       'SR20', '5E', '13B-REW', '2JZ', 'JG1EK', 'C32A1', 'EV20', '119',
       'B230F', '-', '2ZZ', 'G6CT', '5S', 'G6BA', 'AZJ', '112 944', 'FE',
       'G4JS', 'N62B48B', 'K24A1', '_', 'F3BE3681B', '+-', '2JA', 'G4EC',
       'D4GB', '4HK1', 'AD-612T', '없음', '.', '해당 없음'], dtype=object)

In [39]:
# '3304124' : -
# '08784250915' : -
# '09801220246' : -
# '10784203005' : -
# '10784203004' : -
# 'LNS' : -
# '0' : -
# '01821300581' : -
# 'G4AJ' : 휘발유
# 'G15CF' : 휘발유
# 'B5' : 휘발유
# 'H20MR' : 휘발유
# 'G16SF' : 휘발유
# 'MSV3' : 휘발유
# 'ZB' : 경유
# '-' : -
# '1NZ' : 휘발유
# '2AZ' : 휘발유
# 'RW-300' : -
# 'M1082' : -
# '.' : -

# 'DA640' : 경유
# '10-78-11-64609' : -
# '077801946' : 
# 'D0846HM' : 
# '06744200580'
# '10772217991'
# 'DA120'
# '10811142167'
# '04784300942'
# '02791162726'
# '01744500006'
# '01-77-13-00969'
# 'KG-69-C-D-201452'
# 'F-6'
# 'GJS(LJS)'
# '10781153612'
# '03781260209'
# '3.8'
# '0'
# 'G4MBG'
# 'G4AJF045570'
# 'LNS'
# 'CY250909'
# '834C.146'
# 'G4CP'
# '31L'
# '20EV'
# '41.5'
# 'B'
# 'mx'
# '102 962'
# 'VQ30'
# 'K04D3'
# 'V6'
# 'G15MF'
# 'F8A'
# 'CR8.13.1'
# '40S1'
# 'F8C'
# 'B-123'
# '5.0HD'
# 'M28/16'
# 'M30-69'
# 'CRJ'
# 'EFA'
# '104 941'
# '3S-GTE'
# '111 970'
# 'M64/02'
# '117.986'
# '2S25'
# 'SR20'
# '5E'
# '13B-REW'
# '2JZ'
# 'JG1EK'
# 'C32A1'
# 'EV20'
# '119'
# 'B230F'
# '-'
# '2ZZ'
# 'G6CT'
# '5S'
# 'G6BA'
# 'AZJ'
# '112 944'
# 'FE'
# 'G4JS'
# 'N62B48B'
# 'K24A1'
# '_'
# 'F3BE3681B'
# '+-'
# '2JA'
# 'G4EC'
# 'D4GB'
# '4HK1'
# 'AD-612T'

In [22]:
engine_form = ['G4AJ', 'G15CF', 'B5', 'H20MR', 'G16SF', 'MSV3', 'ZB', '1NZ', '2AZ']
len(engine_form)

9

In [23]:
for one in engine_form:
    print(tot2[tot2['엔진형식'] == one].shape)

(5031, 58)
(3235, 58)
(16029, 58)
(2896, 58)
(46, 58)
(1091, 58)
(5834, 58)
(2352, 58)
(1358, 58)


In [24]:
tot2.loc[tot2['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'G15CF', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'B5', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'H20MR', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'G16SF', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'MSV3', '연료'] = '휘발유'
tot2.loc[tot2['엔진형식'] == 'ZB', '연료'] = '경유'
tot2.loc[tot2['엔진형식'] == '1NZ', '연료'] = '경유'
tot2.loc[tot2['엔진형식'] == '2AZ', '연료'] = '경유'

In [53]:
tot2['연료'].value_counts(dropna=False)

휘발유            12138821
경유              9855813
LPG(액화석유가스)     1899359
휘발유 하이브리드       1092740
전기               368150
NaN              146468
경유 하이브리드          35727
CNG(압축천연가스)       30372
수소                28199
LPG 하이브리드         14301
기타연료                926
CNG 하이브리드           304
LNG(액화천연가스)          27
알코올                   3
태양열                   2
등유                    1
Name: 연료, dtype: int64

In [52]:
tot2.loc[tot2['연료'] == '기타연료', '차명'].unique()

array(['미스비시샤시(세미트레일러)', '다이아몬드(추레라)피견인차', '트레라', '세미트레라', '트레일러',
       '세미트레일러', '쎄미트레라', '일산 육륜', '히노 트레라', '피견인차(탱크로리)', '히노탱크로리트레라',
       '히노탱크로리', '코티나마크IV', '후소트레일러', '가와사끼샤시(세미트레일러)', '도큐트레일러',
       '세미 트레일러', '콘테이너샤시콤바인', '콘테이너샤시트레일러', '인터내쇼날트레일러', '추레라',
       '프렛트홈트레일러콤바인', '프렛트홈트레라콤바인', '제너럴트레일러', '아시아AM707중형버스',
       '훌러하프탱크추레라', '후루하프트레라', '드라어  추레라', '현대708콘테이너전용트레일러', '인타담프',
       '동아저상식트레일러', '저상식트레일러', '브리샤2', '추레라(피견인차)', '도큐사요트레러', '새한버스',
       '다이하스웨곤', '스트리크트레일러샤시', '프랫트홈트레일러콤바인', '세미25톤트레일러', '탱크로리트레라',
       '도요다웨곤엠브란스', '도요다크라운', '추례라', '진도콤비네이션콘테너샤시', '쎄미트레일러', '로얄살롱',
       '도요다코로나', '스텔라', '스텔라1.6CXL', '골드호퍼트레일러', '대우11톤카고',
       '현대401드롭배드트레일러', '대우8톤카고', '맥스디젤삼방', '현대리어엔진버스', '대우26톤드롭베스트레일러',
       '포니2', '동아덤프트레일러', '도요다크라운로얄싸롱', '현대구즈넥콘테이너트레일러', '현대평판트레일러',
       '현대20KL스텐탱크트레일러', '23톤구즈넥덤프트레일러', '구즈넥콘테이너샤시', '26톤드톱베드트레일러',
       '구급차(도요다)', '현대콘테이너트레일러', '동아콘테이너샤시', '진도구즈넥콘테이너샤시', '진도프레트폼트레일러',
       '진도20피드콘테이저샤시', '25M3벌크시멘트트레일러', 

In [43]:
set(tot2.loc[tot2['연료'].isnull() == True, '차명'].unique())

{nan,
 '정온쉘터캠핑트레일러',
 '세이프스타싸인보드트레일러',
 '동우8미터평판트레일러',
 'ALTEA 542 캠핑트레일러',
 '투데이3축25톤평판트레일러',
 '한특20피트컨테이너샤시',
 '천지가변형철판운송트레일러',
 'OMK-BS21보트트레일러',
 '케이원24.5kl탱크트레일러',
 '티피글로텍12.5톤차량운송풀트레일러',
 '알리제캠핑트레일러',
 '미래40피트가변형틸팅평판트레일러',
 "성공40'평판트레일러",
 '40평판트레일러',
 '에스알브이이동촬영트레일러',
 '퍼시픽소노마12RB캠핑트레일러',
 'FFB53M^3벌크트레일러',
 '대운캠핑트레일러',
 '선진차량수송세미트레일러',
 '로하카트렌스포터트레일러',
 '대흥25.5톤저상트레일러',
 '카이트알브캠핑트레일러',
 '두성13M 평판트레일러',
 '20톤저상트레일러',
 '대흥 가변식저상트레일러',
 'BK레저장비운송트레일러',
 '지원수난소방장비운송트레일러',
 '우인모빌푸드트레일러',
 '알텍 컨테이너 센터축 트레일러',
 'DA저상슬라이딩 트레일러',
 '유엘피 13톤 가변형 저상트레일러',
 '현대24KL탱크트레일러',
 '뉴에너지레저장비운송트레일러',
 'SM캠핑장비운송트레일러',
 '카막(KAMAG)모듈트레일러',
 'BOECKMANN마필운송트레일러',
 '트리가노(TRIGANO)레저장비운송트레일러',
 '컴퍼트(COMFORTS)이동화장실트레일러',
 '카라반WCE2200보트트레일러',
 '대한40BFLS콤비구즈넥트레일러',
 '미래 40피트 평판트레일러',
 'ETRV넥서스(NAXUS)캠핑트레일러',
 'VOLVO S60',
 '세진18㎥덤프트레일러',
 '대지32㎘탱크트레일러',
 'WSR레저장비운송트레일러',
 '투데이27.5KL액화질소탱크트레일러',
 '케이원23.3톤저상트레일러',
 '13000L가스튜브트레일러',
 'EZ LOADER 보트 트레일러',
 'BUICK LESABRE',
 'SHP3축 저상트레일러',
 '대한25톤가변형평판트레일러',
 '제

In [25]:
tot2.loc[tot2['차명'] == '도요다웨곤엠브란스', '자동차형식']

7031    1980
Name: 자동차형식, dtype: object

In [31]:
tot2.loc[tot2['자동차형식'] == '1980', ['자동차형식', '연료', '차량연식']]

,자동차형식,연료,차량연식
7031,1980,기타연료,2004


In [26]:
tot2.loc[tot2['차명'] == '구급차(도요다)', '자동차형식']

41196    RX60
Name: 자동차형식, dtype: object

In [30]:
tot2.loc[tot2['자동차형식'] == 'RX60', ['자동차형식', '연료']]

,자동차형식,연료
41196,RX60,기타연료


In [27]:
tot2.loc[tot2['차명'] == 'SID 500 F1', '자동차형식']

17724634    C1U52
17726257    C1U52
17726669    C1U52
17726878    C1U52
17727321    C1U52
17727760    C1U52
17728964    C1U52
17728982    C1U52
17729089    C1U52
17730299    C1U52
17731012    C1U52
17731848    C1U52
17733750    C1U52
Name: 자동차형식, dtype: object

In [33]:
tot2.loc[tot2['자동차형식'] == 'C1U52', ['자동차형식', '연료', '차명', ]]

,자동차형식,연료,차명
17724634,C1U52,기타연료,SID 500 F1
17726257,C1U52,기타연료,SID 500 F1
17726669,C1U52,기타연료,SID 500 F1
17726878,C1U52,기타연료,SID 500 F1
17727321,C1U52,기타연료,SID 500 F1
17727760,C1U52,기타연료,SID 500 F1
17728964,C1U52,기타연료,SID 500 F1
17728982,C1U52,기타연료,SID 500 F1
17729089,C1U52,기타연료,SID 500 F1
17730299,C1U52,기타연료,SID 500 F1


In [35]:
car_name = [
    '미스비시샤시(세미트레일러)', #
    '다이아몬드(추레라)피견인차', #
    '트레라', #
    '세미트레라', #
    '트레일러', #
    '세미트레일러', #
    '쎄미트레라', #
    '일산 육륜', 
    '히노 트레라', #
    '피견인차(탱크로리)', #
    '히노탱크로리트레라', #
    '히노탱크로리', # 경유
    '코티나마크IV', # 휘발유
    '후소트레일러', #
    '가와사끼샤시(세미트레일러)', #
    '도큐트레일러', #
    '세미 트레일러', #
    '콘테이너샤시콤바인', #
    '콘테이너샤시트레일러', #
    '인터내쇼날트레일러', #
    '추레라', #
    '프렛트홈트레일러콤바인', #
    '프렛트홈트레라콤바인', #
    '제너럴트레일러', #
    '아시아AM707중형버스', # 경유
    '훌러하프탱크추레라',  #
    '후루하프트레라', #
    '드라어  추레라', #
    '현대708콘테이너전용트레일러', #
    '인타담프', # 경유
    '동아저상식트레일러', #
    '저상식트레일러', #
    '브리샤2', # 휘발유
    '추레라(피견인차)', #
    '도큐사요트레러', #
    '새한버스', # 경유
    '다이하스웨곤', # 휘발유
    '스트리크트레일러샤시', #
    '프랫트홈트레일러콤바인', #
    '세미25톤트레일러', #
    '탱크로리트레라', #

    '도요다웨곤엠브란스', # 경유

    '도요다크라운', # 휘발유
    '추례라', #
    '진도콤비네이션콘테너샤시', #
    '쎄미트레일러', #
    '로얄살롱', # 휘발유
    '도요다코로나', # 휘발유
    '스텔라', # 휘발유
    '스텔라1.6CXL', # 휘발유
    '골드호퍼트레일러', #
    '대우11톤카고', # 경유
    '현대401드롭배드트레일러', #
    '대우8톤카고', # 경유
    '맥스디젤삼방', # 경유
    '현대리어엔진버스', # 경유
    '대우26톤드롭베스트레일러', #
    '포니2', # 휘발유
    '동아덤프트레일러', #
    '도요다크라운로얄싸롱', # 휘발유
    '현대구즈넥콘테이너트레일러', #
    '현대평판트레일러', #
    '현대20KL스텐탱크트레일러', #
    '23톤구즈넥덤프트레일러', #
    '구즈넥콘테이너샤시', #
    '26톤드톱베드트레일러', #

    '구급차(도요다)', # 경유

    '현대콘테이너트레일러',  #
    '동아콘테이너샤시', #
    '진도구즈넥콘테이너샤시', #
    '진도프레트폼트레일러', #
    '진도20피드콘테이저샤시', #
    '25M3벌크시멘트트레일러', #
    "현대45'익스텐션콘테이너트", #
    '벤즈230E', # 휘발유
    '진도구즈넥슬라이더콤비샤시', #
    '프레스토오토매틱', # 휘발유
    '르망펜타-5', # 휘발유
    '진도콤비네이션콘테이너샤시', #
    '스텔라오토매틱', # 휘발유
    '엑셀', # 휘발유
    '벤즈 230E', # 휘발유
    '진도구즈넥슬라이더콘테이너', #
    '동아콘테이너샤시콤바인', #
    '캐피탈', # 휘발유
    '로얄살롱2.0AT', # 휘발유
    '르망', # 휘발유
    '현대45`익스텐션콘테이너트', #
    '현대45,익스텐션콘테이너트', #
    '25M3벌크트레일러', #
    '23M3 벌크트레일러', #
    '머큐리 세이블', # 휘발유
    '트레일러  프렛홈형', # 
    '디엠티 탱크 트레일러', #
    '복사14', # 경유
    'HYUNDAI PRESTO', # 휘발유
    '미래40피트콤비네이션콘테이너샤시', #
    '도요타 프리우스(하이브리드)', # 휘발유 하이브리드
    'TOYOTA CAMRY하이브리드', # 휘발유 하이브리드
    '보트트레일러', # 
    '장보고 보트 트레일러', #
    '동우 가변형 평판트레일러', #
    'XW300', # 
    '대한평판트레일러', #
    '대한콤비컨테이너트레일러', #
    '바인스베르크400LK캠핑트레일러', #
    '미래40피트틸팅컨테이너샤시', #
    '소노마캠팡트레일러', #
    '에어스트림탐지견수송트레일러', #
    '이보1850캠핑트레일러', #
    'RW300', #
    'WSR레저장비운송트레일러', #
    '트레일마스터보트트레일러', #
    '제니스5캠핑트레일러', #
    'ALTEA542캠핑트레일러', #
    '트레일러-LMTV', #
    '카고 트레일러', #
    'TRAILER CARGO LMTV', #
    'D.K트레일러', #
    'FEISHEN일반화물운송트레일러', #

    'SID 500 F1', # 

    '카라반KUS2960평판트레일러', #
    '밤챔버용 트레일러', #
    '아우토니드이동가축관리트레일러', #
    '렉스온캠핑트레일러', #
    '포레스트리버캠핑트레일러', #
    '홍성푸드트레일러', #
    '펜트(FENDT)캠핑트레일러',#
    '밀폐형 방폭트레일러(DynaSEALR X10 LTR on Trailer)', #
    '홍성캠핑트레일러', #
    '아드리아(ADRIA)캠핑트레일러', #
    '트리가노(TRIGANO)스탈렛캠핑트레일러', #
    '엘디스(ELDDIS)캠핑트레일러',#
    '베일리(BAILEY)캠핑트레일러', #
    '바인스버그(WEINSBERG) 캠핑트레일러', #
    '미래40피트평판트레일러', #
    '타버트(TABBERT)캠핑트레일러', #
    '씨티인터내셔널캠핑트레일러', #
]
len(car_name)

138

In [34]:
tot2.loc[tot2['차명'] == '히노탱크로리', '연료'] = '경유'
tot2.loc[tot2['차명'] == '코티나마크IV', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '아시아AM707중형버스', '연료'] = '경유'
tot2.loc[tot2['차명'] == '인타담프', '연료'] = '경유'
tot2.loc[tot2['차명'] == '브리샤2', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '새한버스', '연료'] = '경유'
tot2.loc[tot2['차명'] == '다이하스웨곤', '연료'] = '휘발유'

tot2.loc[tot2['차명'] == '도요다웨곤엠브란스', '연료'] = '경유'

tot2.loc[tot2['차명'] == '도요다크라운', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '로얄살롱', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '도요다코로나', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '스텔라', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '스텔라1.6CXL', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '대우11톤카고', '연료'] = '경유'
tot2.loc[tot2['차명'] == '대우8톤카고', '연료'] = '경유'
tot2.loc[tot2['차명'] == '맥스디젤삼방', '연료'] = '경유'
tot2.loc[tot2['차명'] == '현대리어엔진버스', '연료'] = '경유'
tot2.loc[tot2['차명'] == '포니2', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'

tot2.loc[tot2['차명'] == '구급차(도요다)', '연료'] = '경유'

tot2.loc[tot2['차명'] == '벤즈230E', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '프레스토오토매틱', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '르망펜타-5', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '엑셀', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '벤즈 230E', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '캐피탈', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '로얄살롱2.0AT', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '르망', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '머큐리 세이블', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '복사14', '연료'] = '경유'
tot2.loc[tot2['차명'] == 'HYUNDAI PRESTO', '연료'] = '휘발유'
tot2.loc[tot2['차명'] == '도요타 프리우스(하이브리드)', '연료'] = '휘발유 하이브리드'
tot2.loc[tot2['차명'] == 'TOYOTA CAMRY하이브리드', '연료'] = '휘발유 하이브리드'

## 임시등급 병합

In [42]:
tot2['배출가스등급'].value_counts(dropna=False)

2     10925973
3      8592960
1      2764609
4      1505895
5      1150131
X       288615
T2      246992
T3      135973
T4          37
T5          28
Name: 배출가스등급, dtype: int64

In [44]:
tot2.shape

(25611213, 58)

In [43]:
grade_list = []
for one in tot2['배출가스등급']:
    if str(one) == '1':
        grade_list.append('1')
    elif str(one) == '2' or str(one) == 'T2':
        grade_list.append('2')
    elif str(one) == '3' or str(one) == 'T3':
        grade_list.append('3')
    elif str(one) == '4' or str(one) == 'T4':
        grade_list.append('4')
    elif str(one) == '5' or str(one) == 'T5':
        grade_list.append('5')
    elif str(one) == 'X':
        grade_list.append('X')
    else:
        grade_list.append(np.nan)
len(grade_list)

25611213

In [45]:
grade_list[:5]

['5', '5', '5', '5', '5']

In [46]:
tot2['등급'] = grade_list

## [출력] 등록&제원정보(연료수정, 등급컬럼 추가)

In [53]:
# about 11m 4s
tot2.to_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.29).csv'), index=False, encoding='cp949')

# [로드] 등록&제원정보(연료수정, 등급컬럼 추가)

In [4]:
ods_fold = 'D:/data/big2/ODS'

In [6]:
# about 8m 40s
tot2 = pd.read_csv(os.path.join(ods_fold, '[ODS]등록정보&제원정보(연료수정)(등급컬럼추가)(2022.12.29).csv'), encoding='cp949', low_memory=False)
tot2.shape

(25611213, 59)

## 등급별 중복 차대번호 정리

In [7]:
g1 = tot2[tot2['등급'] == '1']
g2 = tot2[tot2['등급'] == '2']
g3 = tot2[tot2['등급'] == '3']
g4 = tot2[tot2['등급'] == '4']
g5 = tot2[tot2['등급'] == '5']
g1.shape, g2.shape, g3.shape, g4.shape, g5.shape

((2764609, 59), (11172965, 59), (8728933, 59), (1505932, 59), (1150159, 59))

In [8]:
g1 = g1.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g2 = g2.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g3 = g3.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g4 = g4.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g5 = g5.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
g1.shape, g2.shape, g3.shape, g4.shape, g5.shape

((2764594, 59), (11172916, 59), (8728896, 59), (1505892, 59), (1150122, 59))

In [31]:
total = pd.concat([g1, g2, g3, g4, g5], ignore_index=True)
total.shape

(25322420, 59)

## 연료 결측값 샘플

In [32]:
total[total['연료'].isnull() == True].shape

(257, 59)

In [33]:
total.loc[total['연료'].isnull() == True, ['차명', '엔진형식']]

,차명,엔진형식
11998429,CHRYSLER TOWN & COUNTRY(7),NaN
12045462,FORD ESCAPE,NaN
12048481,JEEP GRAND CHEROKEE,NaN
12073761,MERCEDES-BENZ E-CLASS,NaN
12076693,VOLVO XC60,NaN
...,...,...
25321981,GMC,DA120
25322123,기아마스타,NaN
25322257,코티나디럭스,NaN
25322377,GMC,D0846HM


In [34]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [35]:
total.loc[total['연료'].isnull() == True, ['차명', '엔진형식', '차량연식']].drop_duplicates(['차명', '엔진형식', '차량연식'])

,차명,엔진형식,차량연식
11998429,CHRYSLER TOWN & COUNTRY(7),NaN,2010
12045462,FORD ESCAPE,NaN,2010
12048481,JEEP GRAND CHEROKEE,NaN,2010
12073761,MERCEDES-BENZ E-CLASS,NaN,2010
12076693,VOLVO XC60,NaN,2010
...,...,...,...
25321981,GMC,DA120,1976
25322123,기아마스타,NaN,1976
25322257,코티나디럭스,NaN,1974
25322377,GMC,D0846HM,1967


#### [출력] 연료 결측값 샘플(차명, 엔진형식, 차량연식)

In [ ]:
# total.loc[total['연료'].isnull() == True, ['차명', '엔진형식', '차량연식']].drop_duplicates(['차명', '엔진형식']).to_excel('analysis/샘플_연료_결측값_차명_엔진형식_차량연식.xlsx', index=False)

## 연료 결측치 수정

### [로드] 샘플_연료_결측값_차명_엔진형식_차량연식(연료수정)

In [36]:
corrected_fold = 'D:/data/low_gas_vehicle'

In [37]:
corrected_fuel = pd.read_excel(os.path.join(corrected_fold, '샘플_연료_결측값_차명_엔진형식_차량연식(연료수정).xlsx'))
corrected_fuel.shape

(183, 4)

In [38]:
corrected_fuel.head()

,차명,연료,엔진형식,차량연식
0,CHRYSLER TOWN & COUNTRY(7),휘발유,NaN,2010
1,FORD ESCAPE,휘발유,NaN,2010
2,JEEP GRAND CHEROKEE,휘발유,NaN,2010
3,MERCEDES-BENZ E-CLASS,휘발유,NaN,2010
4,VOLVO XC60,휘발유,NaN,2010


In [39]:
corrected_fuel_dict = {a:b for a, b in zip(corrected_fuel['차명'], corrected_fuel['연료'])}

In [40]:
empty_fuel_car_idx = total[total['연료'].isnull() == True].index

In [41]:
len(empty_fuel_car_idx)

257

In [42]:
for i in tqdm(empty_fuel_car_idx):
    temp_car_name = total.loc[i, '차명']
    if temp_car_name in corrected_fuel_dict.keys():
        total.loc[i, '연료'] = corrected_fuel_dict[temp_car_name]

100%|██████████| 257/257 [00:22<00:00, 11.51it/s]


In [43]:
total[total['연료'].isnull() == True].shape

(0, 59)

## 기타 연료 확인

In [44]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [45]:
total.loc[total['연료'] == '기타연료', ['배출가스등급', '차명', '자동차형식','엔진형식', '차량연식', '연료', '시도', '시구군', '읍면동', '리', '지역3']]

,배출가스등급,차명,자동차형식,엔진형식,차량연식,연료,시도,시구군,읍면동,리,지역3
14944440,3,홍성캠핑트레일러,HC-320,.,2020,기타연료,서울,광진구,자양동,NaN,NaN
22669797,T4,에어스트림탐지견수송트레일러,L4DC,-,2007,기타연료,경남,창녕군,계성면,명리,NaN
25322409,T5,일산 육륜,NaN,3304124,1962,기타연료,충북,진천군,백곡면,구수리,NaN


In [66]:
total.loc[total['차명'] == '홍성캠핑트레일러', '연료'] = '무동력'
total.loc[total['차명'] == '에어스트림탐지견수송트레일러', '연료'] = '무동력'
total.loc[total['차명'] == '일산 육륜', '연료'] = '경유'

In [67]:
total.loc[total['연료'] == '기타연료'].shape

(0, 59)

## 본거지법정동코드 확인
- 1대 : 정기, 정밀 검사 데이터(~22.06.30) 없음

In [88]:
total.loc[total['대기관리권역YN'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '차량말소YN', '차량연식', '배출가스등급', '차대번호']]

,본거지법정동코드,시도,시구군,읍면동,리,지역3,차량말소YN,차량연식,배출가스등급,차대번호
25316686,4100000000,경기,NaN,NaN,NaN,NaN,N,1974,5,10742255732


In [89]:
total.loc[total['대기관리권역YN'].isnull() == True, '대기관리권역YN'] = 'Y'

In [90]:
total.loc[total['대기관리권역YN'].isnull() == True].shape

(0, 59)

In [91]:
g5.loc[g5['대기관리권역YN'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '차량말소YN', '차량연식', '배출가스등급', '차대번호']]

,본거지법정동코드,시도,시구군,읍면동,리,지역3,차량말소YN,차량연식,배출가스등급,차대번호
1144388,4100000000,경기,NaN,NaN,NaN,NaN,N,1974,5,10742255732


In [92]:
g5.loc[g5['대기관리권역YN'].isnull() == True, '대기관리권역YN'] = 'Y'

In [94]:
g5.loc[g5['대기관리권역YN'].isnull() == True].shape

(0, 59)

# 분석

##  연료별 통계

In [68]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [69]:
total.groupby(['연료', '등급'], dropna=False)['차대번호'].count()

연료           등급
CNG 하이브리드    1         287
             2          17
CNG(압축천연가스)  2       24227
             3        5989
             4           1
             5           5
LNG(액화천연가스)  2          25
LPG 하이브리드    1       14298
             2           1
LPG(액화석유가스)  1      460966
             2     1168438
             3      235911
             4       26016
             5         435
경유           1        2846
             2         466
             3     7557375
             4     1148951
             5     1119826
경유 하이브리드     3       34139
무동력          3           1
             4           1
             5           5
수소           1       28074
알코올          4           1
             5           1
전기           1      366261
휘발유          1      966489
             2     9839592
             3      895479
             4      330922
             5       29850
휘발유 하이브리드    1      925373
             2      140150
             3           2
Name: 차대번호, dtype: int64

In [70]:
total['등급'].isnull().sum()

0

In [71]:
total[total['등급'] == 'X'].shape

(0, 59)

In [72]:
total.groupby(['연료', '등급'], dropna=False)['차대번호'].count().unstack()

등급,1,2,3,4,5
연료,,,,,
CNG 하이브리드,287.0,17.0,NaN,NaN,NaN
CNG(압축천연가스),NaN,24227.0,5989.0,1.0,5.0
LNG(액화천연가스),NaN,25.0,NaN,NaN,NaN
LPG 하이브리드,14298.0,1.0,NaN,NaN,NaN
LPG(액화석유가스),460966.0,1168438.0,235911.0,26016.0,435.0
경유,2846.0,466.0,7557375.0,1148951.0,1119826.0
경유 하이브리드,NaN,NaN,34139.0,NaN,NaN
무동력,NaN,NaN,1.0,1.0,5.0
수소,28074.0,NaN,NaN,NaN,NaN


In [52]:
total.loc[(total['연료'].isnull() == True) & (total['차명'] == '한원40피트구즈넥트레일러') , ['차명', '차종_등록정보', '차량연식', '자동차형식', '엔진형식', '배출가스등급']]

,차명,차종_등록정보,차량연식,자동차형식,엔진형식,배출가스등급


In [53]:
total.loc[(total['자동차형식'] == '40 GCT') , ['차명', '차종_등록정보', '차량연식', '자동차형식', '엔진형식', '배출가스등급']]

,차명,차종_등록정보,차량연식,자동차형식,엔진형식,배출가스등급
25004311,한원40피트구즈넥트레일러,화물,1999,40 GCT,NaN,5


In [54]:
total.loc[(total['연료'].isnull() == True) & (total['차명'] == '한원40A콤비콘테이너트레일러') , ['차명', '차종_등록정보', '차량연식', '자동차형식', '엔진형식', '배출가스등급']]

,차명,차종_등록정보,차량연식,자동차형식,엔진형식,배출가스등급


In [55]:
total.loc[(total['자동차형식'] == 'HW40ACCT') , ['차명', '차종_등록정보', '차량연식', '자동차형식', '엔진형식', '배출가스등급']]

,차명,차종_등록정보,차량연식,자동차형식,엔진형식,배출가스등급
25007228,한원40A콤비콘테이너트레일러,화물,1999,HW40ACCT,NaN,5
25007239,한원40A콤비콘테이너트레일러,화물,1999,HW40ACCT,NaN,5


In [56]:
total.loc[(total['연료'].isnull() == True) & (total['차명'] == '다임24KL에틸렌글리콜트레일러') , ['차명', '차종_등록정보', '차량연식', '자동차형식', '엔진형식', '배출가스등급']]

,차명,차종_등록정보,차량연식,자동차형식,엔진형식,배출가스등급


In [57]:
total.loc[(total['자동차형식'] == 'DL 24 ETT1') , ['차명', '차종_등록정보', '차량연식', '자동차형식', '엔진형식', '배출가스등급']]

,차명,차종_등록정보,차량연식,자동차형식,엔진형식,배출가스등급
25009753,다임24KL에틸렌글리콜트레일러,화물,1999,DL 24 ETT1,NaN,5


## 5등급 경유 대기관리권역별 통계

In [58]:
g5.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
그 외     강원     48535
        제주     24813
남부권     광주     26354
        전남     77431
동남권     경남    104954
        경북    121512
        대구     50229
        부산     67383
        울산     26339
수도권     경기    225992
        서울    126971
        인천     54924
중부권     대전     26306
        세종      4342
        전북     58835
        충남     62979
        충북     42223
Name: 차대번호, dtype: int64

In [76]:
g5['시도'].isnull().sum()

0

In [78]:
g5['대기관리권역'].value_counts(dropna=False)

수도권    407887
동남권    370417
중부권    194685
남부권    103785
그 외     73348
Name: 대기관리권역, dtype: int64

In [79]:
g5['대기관리권역YN'].value_counts(dropna=False)

Y      866938
N      283183
NaN         1
Name: 대기관리권역YN, dtype: int64

In [80]:
g5.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호',
       '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보',
       '차량말소YN', '최초회원번호', '취득일자', '취득금액', '생성일시_등록정보', '수정일시_등록정보', '차명',
       '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량',
       '총중량', '적재중량', '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명',
       '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', '법정동코드',
       '시도', '시구군', '읍면동', '리', '지역3', '폐지여부', '대기관리권역', '대기관리권역YN', '등급'],
      dtype='object')

In [84]:
g5.loc[g5['대기관리권역YN'].isnull() == True, ['본거지법정동코드', '시도', '시구군', '읍면동', '리', '지역3', '차량말소YN']]

,본거지법정동코드,시도,시구군,읍면동,리,지역3,차량말소YN
1144388,4100000000,경기,NaN,NaN,NaN,NaN,N


In [95]:
g5_y = g5[g5['대기관리권역YN'] == 'Y']
g5_y.shape

(866939, 59)

In [96]:
g5_n = g5[g5['대기관리권역YN'] == 'N']
g5_n.shape

(283183, 59)

In [97]:
g5_y.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
남부권     광주     26354
        전남     32964
동남권     경남     61562
        경북     55251
        대구     50229
        부산     67383
        울산     26339
수도권     경기    216715
        서울    126971
        인천     53716
중부권     대전     26306
        세종      4342
        전북     26900
        충남     60134
        충북     31773
Name: 차대번호, dtype: int64

In [98]:
g5_n.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count()

대기관리권역  시도
그 외     강원    48535
        제주    24813
남부권     전남    44467
동남권     경남    43392
        경북    66261
수도권     경기     9277
        인천     1208
중부권     전북    31935
        충남     2845
        충북    10450
Name: 차대번호, dtype: int64

### [출력] 연료별 통계, 5등급 대기관리권역별 통계

In [99]:
with pd.ExcelWriter('analysis/통계_연료별_5등급대기관리권역별.xlsx') as writer:
    total.groupby(['연료', '등급'], dropna=False)['차대번호'].count().unstack().to_excel(writer, sheet_name='연료별')
    g5_y.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count().to_excel(writer, sheet_name='대기관리권역별_y')
    g5_n.groupby(['대기관리권역', '시도'], dropna=False)['차대번호'].count().to_excel(writer, sheet_name='대기관리권역별_n')